# How to generate error bars for 2Q-GST


In [1]:
import pygsti
import time

#If we were using MPI
# from mpi4py import MPI
# comm = MPI.COMM_WORLD
comm = None

#Load the 2-qubit results (if you don't have this directory, run the 2Q-GST example)
results = pygsti.io.load_results_from_dir("example_files/My2QExample", "StandardGST")

#Set a memory limit
print("Memory limit was = ", results.estimates['CPTP'].parameters.get('memLimit',"none given"))
results.estimates['CPTP'].parameters['memLimit'] = 2.5*(1024.0)**3 # 2.5GB
print("Memory limit is now = ", results.estimates['CPTP'].parameters['memLimit'])

Memory limit was =  3221225472
Memory limit is now =  2684354560.0


In [2]:
# error bars in reports require the presence of a fully-initialized
# "confidence region factory" within the relevant Estimate object.
# In most cases "fully-initialized" means that a Hessian has been 
# computed and projected onto the non-gauge space.
start = time.time()

# initialize a factory for the 'go0' gauge optimization within the 'default' estimate
crfact = results.estimates['CPTP'].add_confidence_region_factory('stdgaugeopt', 'final')
crfact.compute_hessian(comm=comm) #optionally use multiple processors
crfact.project_hessian('intrinsic error')

end = time.time()
print("Total time=%f hours" % ((end - start) / 3600.0))

Evaltree generation (deriv) w/mem limit = 2.47GB
bulk_evaltree: created initial tree (1082 strs) in 1s
bulk_evaltree: split tree (1 subtrees) in 0s
 mem(1 subtrees, 1,1 param-grps, 1 proc-grps) in 1s = 7853.61GB (7853.61GB fc)
Created evaluation tree with 1 subtrees.  Will divide 1 procs into 1 (subtree-processing)
 groups of ~1 procs each, to distribute over (1920,1920) params (taken as 1920,4 param groups of ~1,480 params).
 Memory estimate = 2.03GB (cache=1082, wrtLen1=1, wrtLen2=480, subsPerProc=1).
rank 0: 94.5588s: block 0/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 131.424s: block 1/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 168.311s: block 2/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 204.173s: block 3/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 239.922s: block 4/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 274.424s: block 5/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 309.599s: block 6/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 344.601s: block 7/480

rank 0: 6082.19s: block 115/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6118.64s: block 116/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6154.54s: block 117/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6189.9s: block 118/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6225.23s: block 119/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6261.84s: block 120/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6297.34s: block 121/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6332.55s: block 122/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6367.79s: block 123/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6404.29s: block 124/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6440.17s: block 125/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6475.38s: block 126/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6510.54s: block 127/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6547.22s: block 128/4803, sub-tree 0/1, sub-tree-len = 1082
rank 0: 6582.93s: block 129/4803, sub-tree 0/1, s

KeyboardInterrupt: 

Note above cell was executed for demonstration purposes, and was **keyboard-interrupted intentionally** since it would have taken forever on a single processor.

In [ ]:
#write results back to disk
results.write()